In [1]:
import gurobipy as gp
import numpy as np

In [2]:
inst="1"
P=2 #number of projects
ResCostType=1
core=[]
pred=[]
dur=[]
recu=[]
for i in range(P):
    sample=str(i+1)
    core.append(np.loadtxt("Instancias/rcpsp/Datos30/core"+inst+sample+".txt", dtype='int'))
    pred.append(np.loadtxt("Instancias/rcpsp/Datos30/pred"+inst+sample+".txt", dtype='int'))
    dur.append(np.loadtxt("Instancias/rcpsp/Datos30/dura"+inst+sample+".txt", dtype='int'))
    recu.append(np.loadtxt("Instancias/rcpsp/Datos30/recu"+inst+sample+".txt", dtype='int'))
CmaxAll=np.loadtxt("Cmax.txt")

In [3]:
J=[]
n=[]
H=[]
d=[]
K=0
r=[]
es=[]
ls=[]
Tmax=0
for p in range(P):
    K=max(K,recu[p][-1][0]) 
for p in range(P):
    J.append([dur[p][i][0] for i in range(len(dur[p]))])
    n.append(len(J[p]))
    Hr=pred[p]-1
    d.append([dur[p][i][1] for i in range(len(dur[p]))])    
    r.append([[core[p][4*i+j][2] for j in range(K)] for i in range(n[p])])
    es.append([0 for i in range(n[p])])
    Tmax=Tmax+sum(d[p]) 
    Pr=np.zeros((n[p],n[p]))
    for h in range(len(Hr)):
        Pr[Hr[h,0],Hr[h,1]]=1
    H.append(Pr)
K=int(K)
R=[0 for _ in range(K)]
for k in range(K):
    for p in range(P):        
        R[k]=max(R[k],recu[p][k][1])

Cmax=[]
for p in range(P):
    ls.append([Tmax for i in range(n[p])])

In [4]:
Cmax=[]
for p in range(P):
    Cmax.append(CmaxAll[10*(int(inst)-1)+p])
c=[0 for _ in range(K)]
w=[0 for _ in range(P)]
Rkp=[[0 for _ in range(K)] for _ in range(P)]
for p in range(P):
    for i in range(len(r[p])):
        Rkp[p]=[sum(x) for x in zip(Rkp[p],r[p][i])]
ckp=[[round(10*Rkp[p][k]/min(Rkp[p])) for k in range(K)] for p in range(P)]
for p in range(P):
    w[p]=sum(x*y for x,y in zip(Rkp[p],ckp[p]))
if ResCostType==1:
    c=[10 for k in range(K)]
elif ResCostType==2:
    Rk=[0 for _ in range(K)]
    for p in range(P):
        for k in range(K):
            Rk[k]=Rk[k]+Rkp[p][k]
    c=[round(10*Rk[k]/min(Rkp[p])) for k in range(K)]

In [5]:
for p in range(P):
    Pr=H[p]

    for j in range(n[p]):
        es[p][j] = 0
        for i in range(j):
            if Pr[i,j] == 1:
                if es[p][j] < es[p][i] + d[p][i]:
                    es[p][j] = es[p][i] + d[p][i]

    for i in range(n[p]-1,-1,-1):
        for j in range(i+1,n[p]):
            if Pr[i,j] == 1:
                if ls[p][i] > ls[p][j] - d[p][i]:
                    ls[p][i] = ls[p][j] - d[p][i]

In [6]:
alpha=1   #Cmax=sum(es(np))+alpha*[sum(Cmaxp)-sum(es(np))]
Cmax=max(Cmax)+alpha*(sum(Cmax)-max(Cmax))

In [7]:
RTC=gp.Model("Resource Constrained Project Scheduling Problem")

x=RTC.addVars([(i,p,j,p2) for p in range(P)
                                 for p2 in range(P)
                                 for i in range(n[p])
                                 for j in range(n[p2])], vtype=gp.GRB.BINARY, name='x')
y=RTC.addVars([(i,p,j,p2) for p in range(P)
                                 for p2 in range(P)
                                 for i in range(n[p])
                                 for j in range(n[p2])], vtype=gp.GRB.BINARY, name='y')
S=RTC.addVars([(i,p) for p in range(P)
                        for i in range(n[p])], vtype=gp.GRB.CONTINUOUS, name='S')
z=RTC.addVars(P, vtype=gp.GRB.BINARY, name='h')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


In [8]:
RTC.setObjective(sum(w[p]*z[p] for p in range(P)) - sum(c[k] for k in range(K)), gp.GRB.MAXIMIZE)

RTC.addConstrs((x[i,p,j,p2]+x[j,p2,i,p] <= z[p]  for p in range(P)
                                    for p2 in range(P)
                                    for i in range(n[p])
                                    for j in range(n[p2])
                                    if (H[p][i,j]==0 and p==p2) or p!=p2))
RTC.addConstrs((x[i,p,j,p2]+x[j,p2,i,p] <= z[p2]  for p in range(P)
                                    for p2 in range(P)
                                    for i in range(n[p])
                                    for j in range(n[p2])
                                    if (H[p][i,j]==0 and p==p2) or p!=p2))

RTC.addConstrs((S[j,p2]>= S[i,p] + d[p][i]-Cmax*(1-x[i,p,j,p2]) for p in range(P)
                                                    for p2 in range(P)
                                                    for i in range(n[p])
                                                    for j in range(n[p2])
                                                    if i!=j or p!=p2))

RTC.addConstrs((x[i,p,j,p] == z[p] for p in range(P)
                                for i in range(n[p])
                                for j in range(n[p])
                                if i<j and H[p][i,j]==1))

RTC.addConstrs((S[0,p] == 0 for p in range(P)))
RTC.addConstrs((S[n[p]-1,p] <= Cmax for p in range(P)))
RTC.addConstrs((S[i,p] + d[p][i] <= Cmax for p in range(P)
                                            for i in range(n[p])))

RTC.addConstrs((S[i,p] >= es[p][i]*z[p] for p in range(P)
                                for i in range(n[p])))
RTC.addConstrs((S[i,p] <= ls[p][i]*z[p] for p in range(P)
                                for i in range(n[p])))

RTC.addConstrs((y[i,p,j,p2]<=z[p]-x[i,p,j,p2]-x[j,p2,i,p] for p in range(P)
                                        for p2 in range(P)
                                        for i in range(n[p])
                                        for j in range(n[p2])
                                        if i!=j or p!=p2))

RTC.addConstrs((y[i,p,j,p2]<=z[p] for p in range(P)
                                for p2 in range(P)
                                for i in range(n[p])
                                for j in range(n[p2])
                                if i!=j or p!=p2))
RTC.addConstrs((y[i,p,j,p2]<=z[p2] for p in range(P)
                                for p2 in range(P)
                                for i in range(n[p])
                                for j in range(n[p2])
                                if i!=j or p!=p2))

RTC.addConstrs((S[j,p2]>=S[i,p]-Cmax*(1-y[i,p,j,p2])  for p in range(P)
                                                        for p2 in range(P)
                                                        for i in range(n[p])
                                                        for j in range(n[p2])
                                                        if i!=j or p!=p2))

RTC.addConstrs((S[j,p2]<=S[i,p]+d[p][i]+Cmax*(1-y[i,p,j,p2]) for p in range(P)
                                                            for p2 in range(P)
                                                            for i in range(n[p])
                                                            for j in range(n[p2])
                                                            if i!=j or p!=p2))

RTC.addConstrs((y[i,p,j,p2]+y[j,p2,i,p]+x[i,p,j,p2]+x[j,p2,i,p]>=-1+z[p]+z[p2]  for p in range(P)
                                                for p2 in range(P)
                                                for i in range(n[p])
                                                for j in range(n[p2])
                                                if i!=j or p!=p2))

RTC.addConstrs((r[p][i][k]+sum(r[p][j][k]*y[j,p,i,p] for j in range(n[p]) if j!=i)
                 + sum(sum(r[p2][j][k]*y[j,p2,i,p] for j in range(n[p2]))for p2 in range(P) if p2!=p)
                <=R[k] for p in range(P)
                                                                            for i in range(n[p])
                                                                            for k in range(K)))

RTC.update

<bound method Model.update of <gurobi.Model Continuous instance Resource Constrained Project Scheduling Problem: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>>

In [9]:
RTC.setParam('OutputFlag',True)
RTC.setParam(gp.GRB.Param.TimeLimit,600)
RTC.optimize()
print(RTC.objVal)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 36772 rows, 8258 columns and 118912 nonzeros
Model fingerprint: 0xdf5b30a8
Variable types: 64 continuous, 8194 integer (8194 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -40.0000000
Presolve removed 19985 rows and 1695 columns
Presolve time: 1.72s
Presolved: 16787 rows, 6563 columns, 57598 nonzeros
Variable types: 62 continuous, 6501 integer (6501 binary)

Root relaxation: objective 1.552700e+04, 2722 iterations, 0.07 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incu

In [13]:
print("Objective = ",RTC.objVal)
print("Gap = ",RTC.MIPGap*100)
print("Running time = ",RTC.Runtime)
for v in RTC.getVars():
        if "h" in v.VarName:
            print('%s: %g' % (v.varName, v.x))
for v in RTC.getVars():
    if v.X > 0.1:
        print('%s: %g' % (v.varName, v.x))
print(S)

Objective =  15527.0
Gap =  0.0
Running time =  1.999000072479248
h[0]: 1
h[1]: 1
x[0,0,1,0]: 1
x[0,0,2,0]: 1
x[0,0,3,0]: 1
x[0,0,4,0]: 1
x[0,0,5,0]: 1
x[0,0,6,0]: 1
x[0,0,7,0]: 1
x[0,0,8,0]: 1
x[0,0,9,0]: 1
x[0,0,10,0]: 1
x[0,0,11,0]: 1
x[0,0,12,0]: 1
x[0,0,13,0]: 1
x[0,0,14,0]: 1
x[0,0,15,0]: 1
x[0,0,16,0]: 1
x[0,0,17,0]: 1
x[0,0,18,0]: 1
x[0,0,19,0]: 1
x[0,0,20,0]: 1
x[0,0,21,0]: 1
x[0,0,22,0]: 1
x[0,0,23,0]: 1
x[0,0,24,0]: 1
x[0,0,25,0]: 1
x[0,0,26,0]: 1
x[0,0,27,0]: 1
x[0,0,28,0]: 1
x[0,0,29,0]: 1
x[0,0,30,0]: 1
x[0,0,31,0]: 1
x[1,0,4,0]: 1
x[1,0,5,0]: 1
x[1,0,8,0]: 1
x[1,0,10,0]: 1
x[1,0,11,0]: 1
x[1,0,13,0]: 1
x[1,0,14,0]: 1
x[1,0,15,0]: 1
x[1,0,16,0]: 1
x[1,0,17,0]: 1
x[1,0,18,0]: 1
x[1,0,19,0]: 1
x[1,0,20,0]: 1
x[1,0,21,0]: 1
x[1,0,22,0]: 1
x[1,0,23,0]: 1
x[1,0,24,0]: 1
x[1,0,25,0]: 1
x[1,0,26,0]: 1
x[1,0,27,0]: 1
x[1,0,28,0]: 1
x[1,0,29,0]: 1
x[1,0,30,0]: 1
x[1,0,31,0]: 1
x[2,0,4,0]: 1
x[2,0,5,0]: 1
x[2,0,6,0]: 1
x[2,0,7,0]: 1
x[2,0,8,0]: 1
x[2,0,9,0]: 1
x[2,0,10,0]: 1
x[2,0,